# Decrypting Encrypted Data (From BluePrism)

In [ ]:
pip install pyaes

In [ ]:
#import libraries
import os
import pandas as pd
from pandas import DataFrame
import hashlib
import datetime
import numpy as np
from datetime import datetime

In [ ]:
# #Define input parameters 
container_name, folder_name_encrypted, filename_encrypted = [dbutils.widgets.get(i) for i in ['container_name', 'folder_name_encrypted', 'filename_encrypted']]  

In [ ]:
#Define output parameters
container_name, folder_name_decrypted, filename_decrypted = [dbutils.widgets.get(i) for i in ['container_name', 'folder_name_decrypted', 'filename_decrypted']]  

In [ ]:
#Set the Blob storage
storage_account = 'aihubdatalake'

account_key = dbutils.secrets.get(scope = 'aihubdatalake-key', key = 'aihubdatalake-key')
spark.conf.set(
  f'fs.azure.account.key.{storage_account}.blob.core.windows.net',
  f'{account_key}')

In [ ]:
#Define functions to get files and store files
import os

def wasbs_str(container_name, folder):
  return f'wasbs://{container_name}@{storage_account}.blob.core.windows.net/{folder}/'

def wasbs_file_str(container, folder, file):
  return f'wasbs://{container_name}@{storage_account}.blob.core.windows.net/{folder}/{file}'

def wasbs_folder_str(container, folder):
  return f'wasbs://{container}@{storage_account}.blob.core.windows.net/{folder}/'

def first_object(container, ind, folder):
  return dbutils.fs.ls(wasbs_str(container, folder))[ind][0]

def save_to_storage(df, container_name, filename, folder):
    df.to_parquet(f"/dbfs/FileStore/tables/{filename}")
    return (dbutils.fs.cp(f'/FileStore/tables/{filename}', f'{wasbs_folder_str(container_name, folder)}{filename}'))

spark.conf.set(
  f'fs.azure.account.key.{storage_account}.blob.core.windows.net',
  f'{account_key}')

In [ ]:
#Get the encrypted files from the folder
storage_path = wasbs_file_str(container_name, folder_name_encrypted, filename_encrypted) # wasbs_file_str(container_name, folder_name)
dbutils.fs.cp(storage_path, '.')
local_path=f'/dbfs/{os.path.basename(storage_path)}'
df_filename_encrypted = pd.read_parquet(local_path)

In [ ]:
#Set the BP key from key vault 
storage_account = 'aihubdatalake'

encryption_key = dbutils.secrets.get(scope = 'BP-encryption-key', key = 'BP-encryption-key')
spark.conf.set(
  f'fs.azure.account.key.{storage_account}.blob.core.windows.net',
  f'{account_key}')

In [ ]:
#For P181 Leave Booking we need to get [uniqueid], [status], [loaded], [finished], [data], [attempt], [tag]
#For P181 Travel Dairy Details and P181 Travel Diaries Leave Audit we need to get [uniqueid], [status], [loaded], [finished], [data]
#For P175, P103, P202, P76 we need to get [uniqueid], [data], [loaded]

In [ ]:
#Decrypting data
import pyaes, base64
import pandas as pd

# Decode key from base64
BPkey = encryption_key
BPkey = base64.b64decode(BPkey)

# Load data from CSV file
df = df_filename_encrypted

# Create an empty list to store columns and decrypted data 
decrypted_data = []

# Loop through each row in the dataframe
for index, row in df.iterrows():

    # Extract IV and columns from the 'EncryptedData' column
    iv, queuedata = row['data'].split(':')
    itemid = row['itemid']
    queueid = row['queueid']
    queuename = row['queuename']
    keyfield = row['keyfield']
    keyvalue = row['keyvalue']
    status = row['status']
    attempt = row['attempt']
    tag = row['tag']
    loaded = row['loaded']
    completed = row['completed']
    exception = row['exception']
    exceptionreason = row['exceptionreason']
    deferred = row['deferred']
    finished = row['finished']
    lastupdated = row['lastupdated']
    worktime = row['worktime']
    sessionid = row['sessionid']
    priority = row['priority']
    prevworktime = row['prevworktime']
    attemptworktime = row['attemptworktime']
    exceptionreasonvarchar = row['exceptionreasonvarchar']
    exceptionreasontag = row['exceptionreasontag']
    uniqueid = row['uniqueid']
    ident = ['ident']
    
    # Decode from base64
    iv = base64.b64decode(iv) 
    queuedata = base64.b64decode(queuedata)

    # Decrypt the ciphertext with the given key:
    aes = pyaes.Decrypter(pyaes.AESModeOfOperationCBC(BPkey, iv=iv))
    decrypted = aes.feed(queuedata)
    decrypted += aes.feed()
    decrypted = decrypted.decode("utf-8") 

    # Append the decrypted data along with the columns to the list
    decrypted_data.append((itemid, queueid, queuename, keyfield, keyvalue, status, attempt, tag, loaded, completed, exception, exceptionreason, deferred, finished, lastupdated, worktime, sessionid, priority, prevworktime, attemptworktime, exceptionreasonvarchar, exceptionreasontag, uniqueid, ident, decrypted))

# Create a new dataframe to store the decrypted data and metadata
filename_decrypted_df = pd.DataFrame(decrypted_data, columns=['itemid', 'queueid', 'queuename', 'keyfield', 'keyvalue', 'status', 'attempt', 'tag', 'loaded', 'completed', 'exception', 'exceptionreason', 'deferred', 'finished', 'lastupdated', 'worktime', 'sessionid', 'priority', 'prevworktime', 'attemptworktime', 'exceptionreasonvarchar', 'exceptionreasontag', 'uniqueid', 'ident', 'DecryptedData'])

In [ ]:
#Save decrypted files to Blob
save_to_storage(filename_decrypted_df, container_name, filename_decrypted, folder_name_decrypted)